In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
FOLDERNAME =  "Quadcode/src/ai"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME/
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Quadcode/src/ai
 condacolab_install.log   df_daily_return.csv   env.py	      results	      'trained_models (1)'
 df_actions.csv		  Dow.ipynb	        __pycache__   trained_models   Training.ipynb


#  Portfolio Allocation with SDG alignment of user specific goals


* This is based on  paper: FinRL: A Deep Reinforcement Learning Library for Automated Stock Trading in Quantitative Finance, presented at NeurIPS 2020: Deep RL Workshop.

* It is also based on this article: https://towardsdatascience.com/finrl-for-quantitative-finance-tutorial-for-portfolio-allocation-9b417660c7cd

* **Pytorch Version**



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for portfolio alloacation. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A represents the weight of a stock in the porfolio: a ∈ (-1,1). Assume our stock pool includes N stocks, we can use a list [a<sub>1</sub>, a<sub>2</sub>, ... , a<sub>N</sub>] to determine the weight for each stock in the porfotlio, where a<sub>i</sub> ∈ (-1,1), a<sub>1</sub>+ a<sub>2</sub>+...+a<sub>N</sub>=1. For example, "The weight of AAPL in the portfolio is 10%." is [0.1 , ...].

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [ ]:
## install finrl library
!pip install wrds
!pip install swig
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

✨🍰✨ Everything looks OK!
  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-93y0rk8u
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-93y0rk8u
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit df18dad2f9c3bddabe548f8b516c4798e6a11930
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-zmvow9ic/elegantrl_28ec760c597d4916a3a4e14c3f585612
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-zmvow9ic/elegantrl_28ec760c597d4916a3a4e14c3f585612
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 5ef11a573caeb79d1498119777dfd36bf155e14b
  Preparing metadata (setup.py) ... done



<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them.
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
import datetime

# from finrl import config
# from finrl import config_tickers
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
from finrl.meta.data_processor import DataProcessor
from finrl.meta.data_processors.processor_yahoofinance import YahooFinanceProcessor
import sys
sys.path.append("../FinRL-Library")

/usr/local/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [ ]:

# Get the list of Dow 30 ticker symbols
dow_tickers = [
    "AAPL", "MSFT" "AXP", "BA", "CAT", "CRM", "CSCO", "CVX", "DIS",
    "DOW", "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "KO", "MCD",
    "MMM", "MRK", "MSFT", "NKE", "PG", "TRV", "UNH", "V", "VZ", "WBA", "WMT"
]


<h3> Load Data </h3>

In [ ]:
# Load data
sdg_data = pd.read_csv('../real_data/dow/sdg_data.csv')
sdg_data =sdg_data.rename(columns={"Ticker":"stock"})

user_data = pd.read_csv('../real_data/users.csv')

stock_df = pd.read_csv('../real_data/dow/stock_data.csv')

stock_df.head(5)

,Unnamed: 0,Date,stock,Adj Close,Close,High,Low,Open,Volume
0,0,2009-01-02,AAPL,2.743889,3.241071,3.251429,3.041429,3.067143,746015200.0
1,1,2009-01-02,BA,33.941078,45.250000,45.560001,42.779999,42.799999,7010200.0
2,2,2009-01-02,CAT,30.837599,46.910000,46.980000,44.709999,44.910000,7117200.0
3,3,2009-01-02,CRM,8.493905,8.505000,8.550000,7.912500,8.025000,4069200.0
4,4,2009-01-02,CSCO,11.501413,16.959999,17.000000,16.250000,16.410000,40980600.0


In [ ]:
df = stock_df.merge(sdg_data, on='stock')

In [ ]:
df.head()

,Unnamed: 0,Date,stock,Adj Close,Close,High,Low,Open,Volume,SDG_1,...,SDG_8,SDG_9,SDG_10,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17
0,0,2009-01-02,AAPL,2.743889,3.241071,3.251429,3.041429,3.067143,746015200.0,1,...,1,0,1,0,0,1,0,1,1,1
1,1,2009-01-02,BA,33.941078,45.250000,45.560001,42.779999,42.799999,7010200.0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2009-01-02,CAT,30.837599,46.910000,46.980000,44.709999,44.910000,7117200.0,0,...,0,1,0,1,0,0,0,1,0,0
3,3,2009-01-02,CRM,8.493905,8.505000,8.550000,7.912500,8.025000,4069200.0,0,...,1,0,0,0,0,0,0,0,0,0
4,4,2009-01-02,CSCO,11.501413,16.959999,17.000000,16.250000,16.410000,40980600.0,0,...,0,1,0,0,0,1,0,0,0,0


In [ ]:
df = df.rename(columns={"Date":"date", "stock":"tic"})


In [ ]:
df.columns = [col.lower() for col in df.columns]

In [ ]:
df.shape

(82019, 26)

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [ ]:
fe = FeatureEngineer(use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = False
                    )

df = fe.preprocess_data(df)

Successfully added technical indicators


In [ ]:
df.shape

(81567, 34)

In [ ]:
df.head()

,unnamed: 0,date,tic,adj close,close,high,low,open,volume,sdg_1,...,sdg_16,sdg_17,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,0,2009-01-02,AAPL,2.743889,3.241071,3.251429,3.041429,3.067143,746015200.0,1,...,1,1,0.0,3.502909,3.116019,100.0,66.666667,100.0,3.241071,3.241071
3021,1,2009-01-02,BA,33.941078,45.250000,45.560001,42.779999,42.799999,7010200.0,0,...,0,0,0.0,3.502909,3.116019,100.0,66.666667,100.0,45.250000,45.250000
6042,2,2009-01-02,CAT,30.837599,46.910000,46.980000,44.709999,44.910000,7117200.0,0,...,0,0,0.0,3.502909,3.116019,100.0,66.666667,100.0,46.910000,46.910000
9063,3,2009-01-02,CRM,8.493905,8.505000,8.550000,7.912500,8.025000,4069200.0,0,...,0,0,0.0,3.502909,3.116019,100.0,66.666667,100.0,8.505000,8.505000
12084,4,2009-01-02,CSCO,11.501413,16.959999,17.000000,16.250000,16.410000,40980600.0,0,...,0,0,0.0,3.502909,3.116019,100.0,66.666667,100.0,16.959999,16.959999


In [ ]:
sdg_data = sdg_data.rename(columns={"stock":"tic"})
df = df.merge(sdg_data, on='tic')

In [ ]:
df.head()

,unnamed: 0,date,tic,adj close,close,high,low,open,volume,sdg_1,...,SDG_8,SDG_9,SDG_10,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17
0,0,2009-01-02,AAPL,2.743889,3.241071,3.251429,3.041429,3.067143,746015200.0,1,...,1,0,1,0,0,1,0,1,1,1
1,1,2009-01-02,BA,33.941078,45.250000,45.560001,42.779999,42.799999,7010200.0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2009-01-02,CAT,30.837599,46.910000,46.980000,44.709999,44.910000,7117200.0,0,...,0,1,0,1,0,0,0,1,0,0
3,3,2009-01-02,CRM,8.493905,8.505000,8.550000,7.912500,8.025000,4069200.0,0,...,1,0,0,0,0,0,0,0,0,0
4,4,2009-01-02,CSCO,11.501413,16.959999,17.000000,16.250000,16.410000,40980600.0,0,...,0,1,0,0,0,1,0,0,0,0


## Add covariance matrix as states

In [ ]:
# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values
  cov_list.append(covs)


df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)


In [ ]:
df.shape

(74763, 53)

In [ ]:
df.head()

,unnamed: 0,date,tic,adj close,close,high,low,open,volume,sdg_1,...,SDG_10,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17,cov_list,return_list
0,6804,2010-01-04,AAPL,6.470741,7.643214,7.660714,7.585000,7.622500,493729600.0,1,...,1,0,0,1,0,1,1,1,"[[0.0004430599664845617, 0.0002467199493827335...",tic AAPL BA CAT ...
1,6805,2010-01-04,BA,43.777546,56.180000,56.389999,54.799999,55.720001,6186700.0,0,...,0,0,0,0,0,0,0,0,"[[0.0004430599664845617, 0.0002467199493827335...",tic AAPL BA CAT ...
2,6806,2010-01-04,CAT,40.190220,58.549999,59.189999,57.509998,57.650002,7325600.0,0,...,0,1,0,0,0,1,0,0,"[[0.0004430599664845617, 0.0002467199493827335...",tic AAPL BA CAT ...
3,6807,2010-01-04,CRM,18.680597,18.705000,18.882500,18.547501,18.652500,7906000.0,0,...,0,0,0,0,0,0,0,0,"[[0.0004430599664845617, 0.0002467199493827335...",tic AAPL BA CAT ...
4,6808,2010-01-04,CSCO,16.743505,24.690001,24.840000,24.010000,24.110001,59853700.0,0,...,0,0,0,1,0,0,0,0,"[[0.0004430599664845617, 0.0002467199493827335...",tic AAPL BA CAT ...


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.


## Training data split: 2009-01-01 to 2020-07-01

In [ ]:
train = data_split(df, '2009-01-01', '2019-12-31',)
trade = data_split(df, '2020-01-01', '2021-01-01')

In [ ]:
train.head()

,unnamed: 0,date,tic,adj close,close,high,low,open,volume,sdg_1,...,SDG_10,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17,cov_list,return_list
0,6804,2010-01-04,AAPL,6.470741,7.643214,7.660714,7.585000,7.622500,493729600.0,1,...,1,0,0,1,0,1,1,1,"[[0.0004430599664845617, 0.0002467199493827335...",tic AAPL BA CAT ...
0,6805,2010-01-04,BA,43.777546,56.180000,56.389999,54.799999,55.720001,6186700.0,0,...,0,0,0,0,0,0,0,0,"[[0.0004430599664845617, 0.0002467199493827335...",tic AAPL BA CAT ...
0,6806,2010-01-04,CAT,40.190220,58.549999,59.189999,57.509998,57.650002,7325600.0,0,...,0,1,0,0,0,1,0,0,"[[0.0004430599664845617, 0.0002467199493827335...",tic AAPL BA CAT ...
0,6807,2010-01-04,CRM,18.680597,18.705000,18.882500,18.547501,18.652500,7906000.0,0,...,0,0,0,0,0,0,0,0,"[[0.0004430599664845617, 0.0002467199493827335...",tic AAPL BA CAT ...
0,6808,2010-01-04,CSCO,16.743505,24.690001,24.840000,24.010000,24.110001,59853700.0,0,...,0,0,0,1,0,0,0,0,"[[0.0004430599664845617, 0.0002467199493827335...",tic AAPL BA CAT ...


In [ ]:
user_data.head()

,Unnamed: 0,Username,Desired_Risk,Age,Investment_Amount,SDG_1,SDG_2,SDG_3,SDG_4,SDG_5,...,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17,ESG_1,ESG_2,ESG_3
0,0,User_1,Low,52,16262.172067,0.0,0.0,0.0,0.2,0.0,...,0.2,0.0,0.2,0.0,0.0,0.2,0.0,0,0,1
1,1,User_2,High,54,84829.915374,0.0,0.0,0.2,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0,1,0
2,2,User_3,Medium,25,32823.605854,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.2,0.2,0.0,0.0,0,1,0
3,3,User_4,Low,64,56007.128253,0.0,0.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.2,0.0,0.0,0.0,1,0,0
4,4,User_5,Medium,52,54512.068564,0.0,0.0,0.0,0.2,0.0,...,0.2,0.2,0.0,0.0,0.2,0.2,0.0,1,0,0


In [ ]:
train.to_csv("targeted_df.csv")

Exception ignored in: <function _xla_gc_callback at 0x790eff102a70>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
train.loc[0,:]

,unnamed: 0,date,tic,adj close,close,high,low,open,volume,sdg_1,...,SDG_10,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17,cov_list,return_list
0,40770,2015-01-02,AAPL,24.435270,27.332500,27.860001,26.837500,27.847500,212818400.0,1,...,1,0,0,1,0,1,1,1,"[[0.00018602788872971516, 4.19231079351529e-05...",tic AAPL BA CAT ...
0,40771,2015-01-02,BA,113.657204,129.949997,131.839996,129.089996,131.070007,4294200.0,0,...,0,0,0,0,0,0,0,0,"[[0.00018602788872971516, 4.19231079351529e-05...",tic AAPL BA CAT ...
0,40772,2015-01-02,CAT,70.907600,91.879997,92.370003,90.660004,91.769997,3767900.0,0,...,0,1,0,0,0,1,0,0,"[[0.00018602788872971516, 4.19231079351529e-05...",tic AAPL BA CAT ...
0,40773,2015-01-02,CRM,59.162720,59.240002,60.430000,58.509998,59.900002,2796400.0,0,...,0,0,0,0,0,0,0,0,"[[0.00018602788872971516, 4.19231079351529e-05...",tic AAPL BA CAT ...
0,40774,2015-01-02,CSCO,20.657427,27.610001,28.120001,27.379999,27.860001,22926500.0,0,...,0,0,0,1,0,0,0,0,"[[0.00018602788872971516, 4.19231079351529e-05...",tic AAPL BA CAT ...
0,40775,2015-01-02,CVX,75.044243,112.580002,113.000000,110.849998,111.629997,5898800.0,0,...,0,0,0,1,0,0,0,0,"[[0.00018602788872971516, 4.19231079351529e-05...",tic AAPL BA CAT ...
0,40776,2015-01-02,DIS,87.092415,93.750000,95.279999,92.849998,94.910004,5865400.0,0,...,0,0,0,0,0,0,0,0,"[[0.00018602788872971516, 4.19231079351529e-05...",tic AAPL BA CAT ...
0,40777,2015-01-02,GS,161.304291,194.410004,195.729996,192.699997,195.300003,1877700.0,0,...,0,0,0,0,0,0,0,0,"[[0.00018602788872971516, 4.19231079351529e-05...",tic AAPL BA CAT ...
0,40778,2015-01-02,HD,82.825241,103.430000,105.660004,102.669998,105.160004,4323700.0,0,...,0,0,0,0,0,0,0,0,"[[0.00018602788872971516, 4.19231079351529e-05...",tic AAPL BA CAT ...
0,40779,2015-01-02,HON,78.520927,95.556229,96.147316,94.478920,95.527626,2232817.0,0,...,0,1,0,0,0,0,0,0,"[[0.00018602788872971516, 4.19231079351529e-05...",tic AAPL BA CAT ...


In [ ]:
train.index

Index([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       ...
       1256, 1256, 1256, 1256, 1256, 1256, 1256, 1256, 1256, 1256],
      dtype='int64', length=33939)

## Environment for Portfolio Allocation


In [ ]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step


    """
    metadata = {'render.modes': ['human']}

    def __init__(self,
                df,
                user_df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                 profit_weight,
                 sdg_weight,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.user_df = user_df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,))
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list)+17+17,self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        self.turbulence_threshold = turbulence_threshold
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount



        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        self.user_sdg_preferences = self.user_df.sample(n=1, random_state=42)[['SDG_' +str(i) for i in range(1,18)]].values

        self.profit_weight= profit_weight
        self.sdg_weight = sdg_weight


    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()

            df = pd.DataFrame(self.social_return_memory)
            df.columns = ['social_return']
            plt.plot(df.social_return.cumsum(),'s')
            plt.savefig('results/social_reward.png')
            plt.close()

            plt.plot(self.social_return_memory,'s')
            plt.savefig('results/social_rewards.png')
            plt.close()

            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/portfolio_rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_social = pd.DataFrame(self.social_return_memory)

            df_daily_return.columns = ['daily_return']
            df_daily_social.columns = ['daily_social']

            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("social:",  df_daily_social['daily_social'].mean()*5)
            print("=================================")

            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions)
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state = self.get_state() #np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            #print(self.state)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            #self.reward = self.profit_weight * new_portfolio_value + self.sdg_weight * social_impact

            social_impact= self.calculate_social_alignment(weights)[0]
            self.social_return_memory.append(social_impact)

             # the reward is the new portfolio value or end portfolo value
            self.reward = self.profit_weight * new_portfolio_value + self.sdg_weight * social_impact *100000

            #print("Step reward: ", self.reward)
            #self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def get_state(self):
        obs = np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        obs = np.append(obs, [self.data[tech].values.tolist() for tech in ['SDG_' +str(i) for i in range(1,18)] ],  axis=0)
        duplicated_array = np.tile(self.user_sdg_preferences, (27,1))
        # print(obs.shape)
        # print(duplicated_array.shape)

        obs = np.append(obs, duplicated_array.T,  axis=0)
        return obs

    def calculate_social_alignment(self,weights ):
        sdg_scores = self.data[['SDG_' +str(i) for i in range(1,18)]].values
        sdg_alignment = np.dot(sdg_scores, np.array(self.user_sdg_preferences).reshape(17, -1))


        weighted_sdgs_score = np.matmul( weights, sdg_alignment)

        return  weighted_sdgs_score

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  self.get_state() #np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.social_return_memory =[0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]
        return self.state

    def render(self, mode='human'):
        return self.state

    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output


    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']

        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

**<h3> SDG/Social Alignment of Portfolio Allocation to User SDG preferences to balance profit with social impact of User specific goals. </h3>**
<p> Note the the calculate_social_alignment in the env.
<p> It calculates a single value of User SDG alignment between User SDG vector and assets SDG contributions vector by dot products.
<p> Essentially this is part of the reward( along with portfolio value) to favour assets that contribute to User SDG goals .


In [ ]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 27, State Space: 27


In [ ]:
indicators = [
    'macd',
    'boll_ub',
    'boll_lb',
    'rsi_30',
    'cci_30',
    'dx_30',
    'close_30_sma',
    'close_60_sma'
]

In [ ]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "transaction_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": indicators,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    "profit_weight": 0.5,
        "sdg_weight": 0.5,

}

e_train_gym = StockPortfolioEnv(df = train, user_df=user_data, **env_kwargs)

In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [ ]:
# initialize
agent = DRLAgent(env = env_train)

### Model 1: **A2C**


In [ ]:
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cpu device


In [ ]:
trained_a2c = agent.train_model(model=model_a2c,
                                tb_log_name='a2c',
                                total_timesteps=50000)

-------------------------------------
| time/                 |           |
|    fps                | 123       |
|    iterations         | 100       |
|    time_elapsed       | 4         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -38.2     |
|    explained_variance | 0         |
|    learning_rate      | 0.0002    |
|    n_updates          | 99        |
|    policy_loss        | 1.34e+08  |
|    reward             | 1172502.1 |
|    std                | 0.997     |
|    value_loss         | 1.46e+13  |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 138       |
|    iterations         | 200       |
|    time_elapsed       | 7         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -38.2     |
|    explained_variance | 1.19e-07  |
|    learning_rate      | 0.0002    |
|    n_updat

In [ ]:
trained_a2c.save('trained_models/trained_a2c.zip')

### Model 2: **PPO**


In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 7048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 7048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
Using cpu device


/usr/local/lib/python3.10/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 128, but because the `RolloutBuffer` is of size `n_steps * n_envs = 7048`, after every 55 untruncated mini-batches, there will be a truncated mini-batch of size 8
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=7048 and n_envs=1)
  warnings.warn(


In [ ]:
env_train

In [ ]:
trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=180000)

begin_total_asset:1000000
end_total_asset:3831847.4155087047
Sharpe:  1.0187409569883423
social: 0.11846421000779149
begin_total_asset:1000000
end_total_asset:3450595.6706553046
Sharpe:  0.9449651712423827
social: 0.11819492251214167
----------------------------------
| time/              |           |
|    fps             | 200       |
|    iterations      | 1         |
|    time_elapsed    | 35        |
|    total_timesteps | 7048      |
| train/             |           |
|    reward          | 1185.6797 |
----------------------------------
begin_total_asset:1000000
end_total_asset:3673554.52816849
Sharpe:  0.9942382929973543
social: 0.1188829035742975
begin_total_asset:1000000
end_total_asset:3607989.555374517
Sharpe:  0.9821847568324581
social: 0.11871051920663056
begin_total_asset:1000000
end_total_asset:4037976.818619285
Sharpe:  1.0622160209342961
social: 0.11906559796622451
-----------------------------------------
| time/                   |             |
|    fps             

In [ ]:
trained_ppo.save('trained_models/trained_ppo.zip')

In [ ]:
indicators

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [ ]:
cov_list = trade.loc[0,:].cov_list.values[0]
tech_indicator_list = trade.loc[0,:][indicators].values

In [ ]:
cov_list.shape

(27, 27)

In [ ]:
tech_indicator_list.shape

(27, 8)

### Model 3: **DDPG**


In [ ]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}


model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device


In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg,
                             tb_log_name='ddpg',
                             total_timesteps=50000)

begin_total_asset:1000000
end_total_asset:1773358.6888129262
Sharpe:  0.9368314091238091
social: 0.45912646142026453


KeyboardInterrupt: 

In [ ]:
trained_ddpg.save('/content/trained_models/trained_ddpg.zip')

### Model 4: **SAC**


In [ ]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0003,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

In [ ]:
trained_sac = agent.train_model(model=model_sac,
                             tb_log_name='sac',
                             total_timesteps=50000)

In [ ]:
trained_sac.save('/content/trained_models/trained_sac.zip')

### Model 5: **TD3**


In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100,
              "buffer_size": 1000000,
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

In [ ]:
trained_td3 = agent.train_model(model=model_td3,
                             tb_log_name='td3',
                             total_timesteps=30000)

In [ ]:
trained_td3.save('/content/trained_models/trained_td3.zip')

## Trading
Assume that we have $1,000,000 initial capital at 2019-01-01. We use the PPO model to trade Dow jones 30 stocks.

In [ ]:
#trade = data_split(df,'2020-07-01', '2021-10-31')
e_trade_gym = StockPortfolioEnv(df = trade, user_df=user_data, **env_kwargs)


In [ ]:
e_trade_gym.get_state().shape

(69, 27)

In [ ]:
trade.shape

<h3> Projected portfolio value, sharpe ratio and social alignment of SDGs

In [ ]:
df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_ppo,
                        environment = e_trade_gym)

/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:1000000
end_total_asset:1122421.2115457556
Sharpe:  0.5039629677972853
social: 0.7371763018928026
hit end!


In [ ]:
df_daily_return.head()

,date,daily_return
0,2020-01-02,0.000000
1,2020-01-03,-0.008726
2,2020-01-06,0.002919
3,2020-01-07,-0.005400
4,2020-01-08,0.003779


In [ ]:
df_daily_return.to_csv('df_daily_return.csv')

In [ ]:
df_actions.tail()

,AAPL,BA,CAT,CRM,CSCO,CVX,DIS,GS,HD,HON,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2020-12-24,0.069598,0.033291,0.052131,0.033291,0.036475,0.036951,0.033291,0.033291,0.033291,0.034707,...,0.033291,0.050400,0.033291,0.036526,0.033291,0.033291,0.033291,0.033291,0.033291,0.036151
2020-12-28,0.069320,0.033324,0.051977,0.033324,0.036493,0.037006,0.033324,0.033324,0.033324,0.034775,...,0.033324,0.050163,0.033324,0.036501,0.033324,0.033324,0.033324,0.033324,0.033324,0.036164
2020-12-29,0.067185,0.033542,0.051289,0.033542,0.036505,0.036875,0.033542,0.033542,0.033542,0.034807,...,0.033542,0.049318,0.033542,0.036705,0.033542,0.033542,0.033542,0.033542,0.033542,0.036098
2020-12-30,0.067892,0.033463,0.051675,0.033463,0.036346,0.036772,0.033463,0.033463,0.033463,0.034885,...,0.033463,0.049583,0.033463,0.036735,0.033463,0.033463,0.033463,0.033463,0.033463,0.036087
2020-12-31,0.067799,0.033473,0.051645,0.033473,0.036344,0.036760,0.033473,0.033473,0.033473,0.034888,...,0.033473,0.049554,0.033473,0.036749,0.033473,0.033473,0.033473,0.033473,0.033473,0.036083


In [ ]:
df_actions.iloc[100]

AAPL    0.071527
BA      0.033112
CAT     0.052824
CRM     0.033112
CSCO    0.036220
CVX     0.036897
DIS     0.033112
GS      0.033112
HD      0.033112
HON     0.034724
IBM     0.036590
INTC    0.033112
JNJ     0.037261
JPM     0.037598
KO      0.033112
MCD     0.033112
MMM     0.036398
MRK     0.033112
MSFT    0.050686
NKE     0.033112
PG      0.036492
TRV     0.033112
UNH     0.033112
V       0.033112
VZ      0.033112
WBA     0.033112
WMT     0.036101
Name: 2020-05-27, dtype: float64

In [ ]:
df_actions.to_csv('df_actions.csv')

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
from pyfolio import timeseries
DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
perf_func = timeseries.perf_stats
perf_stats_all = perf_func( returns=DRL_strat,
                              factor_returns=DRL_strat,
                                positions=None, transactions=None, turnover_denom="AGB")

In [ ]:
print("==============DRL Strategy Stats===========")
perf_stats_all

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI",
        start = df_daily_return.loc[0,'date'],
        end = df_daily_return.loc[len(df_daily_return)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
import pyfolio
%matplotlib inline

baseline_df = get_baseline(
        ticker='^DJI', start=df_daily_return.loc[0,'date'], end='2021-11-01'
    )

baseline_returns = get_daily_return(baseline_df, value_col_name="close")

with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(returns = DRL_strat,
                                       benchmark_rets=baseline_returns, set_context=False)

## Min-Variance Portfolio Allocation

In [ ]:
%pip install PyPortfolioOpt

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models

In [ ]:
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

In [ ]:
df.head()

In [ ]:
#calculate_portfolio_minimum_variance
portfolio = pd.DataFrame(index = range(1), columns = unique_trade_date)
initial_capital = 1000000
portfolio.loc[0,unique_trade_date[0]] = initial_capital

for i in range(len( unique_trade_date)-1):
    df_temp = df[df.date==unique_trade_date[i]].reset_index(drop=True)
    df_temp_next = df[df.date==unique_trade_date[i+1]].reset_index(drop=True)
    #Sigma = risk_models.sample_cov(df_temp.return_list[0])
    #calculate covariance matrix
    Sigma = df_temp.return_list[0].cov()
    #portfolio allocation
    ef_min_var = EfficientFrontier(None, Sigma,weight_bounds=(0, 0.1))
    #minimum variance
    raw_weights_min_var = ef_min_var.min_volatility()
    #get weights
    cleaned_weights_min_var = ef_min_var.clean_weights()

    #current capital
    cap = portfolio.iloc[0, i]
    #current cash invested for each stock
    current_cash = [element * cap for element in list(cleaned_weights_min_var.values())]
    # current held shares
    current_shares = list(np.array(current_cash)
                                      / np.array(df_temp.close))
    # next time period price
    next_price = np.array(df_temp_next.close)
    ##next_price * current share to calculate next total account value
    portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)

portfolio=portfolio.T
portfolio.columns = ['account_value']

In [ ]:
portfolio.head()

In [ ]:
a2c_cumpod =(df_daily_return.daily_return+1).cumprod()-1

In [ ]:
min_var_cumpod =(portfolio.account_value.pct_change()+1).cumprod()-1

In [ ]:
dji_cumpod =(baseline_returns+1).cumprod()-1

## Plotly: DRL, Min-Variance, DJIA

In [ ]:
%pip install plotly

In [ ]:
from datetime import datetime as dt

import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go

In [ ]:
time_ind = pd.Series(df_daily_return.date)

In [ ]:
trace0_portfolio = go.Scatter(x = time_ind, y = a2c_cumpod, mode = 'lines', name = 'PPO (Portfolio Allocation)')

trace1_portfolio = go.Scatter(x = time_ind, y = dji_cumpod, mode = 'lines', name = 'DJIA')
trace2_portfolio = go.Scatter(x = time_ind, y = min_var_cumpod, mode = 'lines', name = 'Min-Variance')
#trace3_portfolio = go.Scatter(x = time_ind, y = ddpg_cumpod, mode = 'lines', name = 'DDPG')
#trace4_portfolio = go.Scatter(x = time_ind, y = addpg_cumpod, mode = 'lines', name = 'Adaptive-DDPG')
#trace5_portfolio = go.Scatter(x = time_ind, y = min_cumpod, mode = 'lines', name = 'Min-Variance')

#trace4 = go.Scatter(x = time_ind, y = addpg_cumpod, mode = 'lines', name = 'Adaptive-DDPG')

#trace2 = go.Scatter(x = time_ind, y = portfolio_cost_minv, mode = 'lines', name = 'Min-Variance')
#trace3 = go.Scatter(x = time_ind, y = spx_value, mode = 'lines', name = 'SPX')

In [ ]:
fig = go.Figure()
fig.add_trace(trace0_portfolio)

fig.add_trace(trace1_portfolio)

fig.add_trace(trace2_portfolio)



fig.update_layout(
    legend=dict(
        x=0,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=15,
            color="black"
        ),
        bgcolor="White",
        bordercolor="white",
        borderwidth=2

    ),
)
#fig.update_layout(legend_orientation="h")
fig.update_layout(title={
        #'text': "Cumulative Return using FinRL",
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
#with Transaction cost
#fig.update_layout(title =  'Quarterly Trade Date')
fig.update_layout(
#    margin=dict(l=20, r=20, t=20, b=20),

    paper_bgcolor='rgba(1,1,0,0)',
    plot_bgcolor='rgba(1, 1, 0, 0)',
    #xaxis_title="Date",
    yaxis_title="Cumulative Return",
xaxis={'type': 'date',
       'tick0': time_ind[0],
        'tickmode': 'linear',
       'dtick': 86400000.0 *80}

)
fig.update_xaxes(showline=True,linecolor='black',showgrid=True, gridwidth=1, gridcolor='LightSteelBlue',mirror=True)
fig.update_yaxes(showline=True,linecolor='black',showgrid=True, gridwidth=1, gridcolor='LightSteelBlue',mirror=True)
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='LightSteelBlue')

fig.show()